In [2]:
# 개와 고양이의 이미지 데이터 불러오기
import tensorflow_datasets as tfds

C:\Users\COMPUTER\anaconda3\envs\tf2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
pip install tensorflow_datasets

Note: you may need to restart the kernel to use updated packages.


In [4]:
tfds.disable_progress_bar()

In [6]:
raw_train, raw_test = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[20%:]'],
    as_supervised=True
)

In [17]:
#  개와 고양이 이미지 데이터 전처리
import numpy as np
import tensorflow as tf

def preprocess(image, label):
    out_image = tf.image.resize(image/255, [224, 224])
    return out_image, label

batch_size = 32
train_batch = raw_train.map(preprocess).batch(batch_size)
test_batch = raw_test.map(preprocess).batch(batch_size)

In [12]:
# 모델 생성 및 평가
!pip install tensorflow_hub

In [13]:
# 텐서플로 허브 모델 생성
import tensorflow_hub as hub

url = "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4"
#  https://www.kaggle.com/api/v1/models/google/mobilenet-v2/tensorFlow2/100-224-feature-vector/2/download
hub_model_transfer = tf.keras.Sequential([
    hub.KerasLayer(url, input_shape=(224, 224, 3), trainable=False),
    tf.keras.layers.Dense(1)
])

In [15]:
# 텐서플로 허브 모델 컴파일 및 학습
hub_model_transfer.compile(optimizer='adam',
                           loss='binary_crossentropy',
                           metrics=['accuracy'])
hub_model_transfer.fit(train_batch, epochs=5)

Epoch 1/5
582/582 [==============================] - 272s 448ms/step - loss: 0.2639 - accuracy: 0.9709
Epoch 2/5
582/582 [==============================] - 259s 445ms/step - loss: 0.2636 - accuracy: 0.9759
Epoch 3/5
582/582 [==============================] - 259s 445ms/step - loss: 0.2058 - accuracy: 0.9808
Epoch 4/5
582/582 [==============================] - 254s 437ms/step - loss: 0.1558 - accuracy: 0.9867
Epoch 5/5
582/582 [==============================] - 255s 438ms/step - loss: 0.1727 - accuracy: 0.9848


In [18]:
# 텐서플로 허브 모델 평가
hub_model_transfer.evaluate(test_batch, verbose=2)

582/582 - 256s - loss: 0.1387 - accuracy: 0.9866 - 256s/epoch - 440ms/step


[0.1387164145708084, 0.9865663647651672]